### Create a notebook to perform Runge-Kutta integration for multiple coupled variables

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

### Define our coupled derivativs to integrate

In [ ]:
def dydx(x,y):
        
    # set the derivatives
        
    # our equation is d^2x/dx^2 = -y
        
    # so we can write
    # dydx = z
    # dzdx = -y
        
    # we will set y = y[0]   (the function y)
    # we will set z = y[1]   (the function x)
        
    # declare an array
    y_derivs = np.zeros(2)  # array of functions
        
    # set dydx = z
    y_derivs[0] = y[1]      # dydx we put in y_derivs[1]
        
    # set dydx = -y
    y_derivs[1] = -1*y[0]
        
    # here we have to return an array
    return y_derivs

### Define the 4th order RK method

In [ ]:
# now we'll operate on all the elements at the same time 
def rk4_mv_core(dydx,xi,yi,nv,h):   #dydx is function we just wrote, 
                                    # xi is value of x at step i
                                    # xi changes by value of h
                                    # nv is number of variables
                                    # yi is the array 
    # declare k? arrays
    k1 = np.zeros(nv)               # each k is array of two emelents one that
    k2 = np.zeros(nv)               # corresponds to y and one that corresponds to z
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    # define x at 1/2 step
    x_ipoh = xi + 0.5*h
    
    # define x at 1 step
    x_ipo = xi + h
    
    # declare a temp y array, will contain estimated values of y and z
    # ad different points of steps
    y_temp = np.zeros(nv)
    
    # get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    # get k2 values
    y_temp[:] = yi[:] + 0.5*k1[:]    # initial values on left side + .5 of k1, we get to midpoint
    y_derivs = dydx(x_ipoh,y_temp)   # estimate of y and z at half step and 
    k2[:] = h*y_derivs[:]            # recompute derivatives at half step
    
    # get k3 values
    y_temp[:] = yi[:] + 0.5*k2[:]    # different step, with different derivative
    y_derivs = dydx(x_ipoh,y_temp)   # retaking half step and recomputing derivatives
    k3[:] = h*y_derivs[:]            # 
    
    # get k4 values
    y_temp[:] = yi[:] + k3[:]        # taking full step for k4 using recomputed
    y_derivs = dydx(x_ipoh,y_temp)   # derivatives from above
    k4[:] = h*y_derivs[:]
    
    # advance y by a step h
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.
    
    return yipo

### Define an adaptive step size driver for RK4

In [ ]:
# dydx functions that take drivatives
# x_i value of x at step i
# y_i values of items in array at i
# h is step size
# tol tolerance
# this function  
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY    = 0.9
    H_NEW_FAC = 2.0
    
    # set a maximum number of iterations
    imax = 10000
    
    # set an iteration variable 
    i = 0
    
    # create an error
    Delta = np.full(nv,2*tol)    # array that contains error estimates
    
    # remember the step
    h_step = h
    
    # adjust steps until error is in our tolerance
    while(Delta.max()/tol > 1.0):
        
        #print(Delta.max(),h,x_i,y_i,nv,h_step)
       
        
        # estimate our error by taking one step of size h
        # vs. two steps of size h/2
        y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)        # copmaring to one full step
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)    # copmaring to two steps of h/2
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)    # using y_1 estimate here which is 
                                                                    # estimate for y and z from previous step
                                                                    # that used half step
        
        # compute an error
        Delta = np.fabs(y_2 - y_11)
        
        # if the error is too large, take a smaller step
        # Delta.max() is the biggest element in Delta
        # if this value is > 1 after divided by tol, our step was too big
        if(Delta.max()/tol > 1.0):
            
            # our errors is too large, decrease the step 
            # multiplying by SAFETY to get smaller step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        # check iteration
        if(i >= imax):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ", i)
            
        # iterate
        i+=1
        
    # next time, try to take a bigger step, so estimate new h step with below
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    # return the answer, a new step, and the step we actually took
    return y_2, h_new, h_step

### Define a wrapper for RK4

In [ ]:
# this function, you pass in the derivatives you want to evolve
# the starting place, the ending place, and tolerance
# it will call the two functions we just wrote 
# we don't know how many steps we'll take so we have initial conditions
# we just provide some tolerance and want for this wrapper to try to stay
# within the tolerance

def rk4_mv(dydx,a,b,y_a,tol):
    
    # dydx  is the derivate wrt x
    # a  is the lower bound
    # b  is the upper bound
    # y_a  are the boundary conditions
    # tol  is the tolerance for integrating y
    
    # define our starting step
    xi = a              # current value of x 
    yi = y_a.copy()     # current value of 
    
    # an initial step size == make very small
    h = 1.0e-4 * (b-a) 
    
    # set a minimum number of iterations since we don't know how
    # many iterations we'll need to take
    imax = 10000
    
    # set an iteration variable
    i = 0
    
    # set the number of coupled odes to the size of y_a
    nv = len(y_a)     # initial comditions of y_a
    
    # set the initial conditions ( arrays that we'll be plotting )
    x = np.full(1,a)            #  signle element array with value a
    y = np.full((1,nv),y_a)     #  array with all values of x at all nv steps, 
                                #  and y and z at values of x (variable y is actually
                                #  an array)
    
    # set a flag
    flag = 1
    
    # loop until we reach the right side
    while(flag):
        
        # calculate y_i+1
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        
        # update the step
        h = h_new 
        
        # prevent an overshoot, as we integrate along x, when we get close to
        # edge, we don't want to cross it, so we retake step to get to the edge
        if(xi+h_step>b):
            
            # take a smaller step
            h = b-xi
            
            # recalculate y_i+1
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            
            # break
            flag = 0
            
        # update values in the arrays
        xi += h_step
        yi[:] = yi_new[:]   # arrays
            
        # add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y  # frees memory of y
        y = y_new
            
        # prevent too many iterations
        if(i>=imax):
                
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ", i)
                
        #iterate 
        i += 1
            
        # output some information
        # %3d is a format of the integer so here we want 3 places in the integer
        # \t means to pritn out tab space on screen 
        # %9.8f print out floating number with 8 digets on right hand side of decimal
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi,h_step,b)
        print(s)
            
        # break if new xi is == b
        if(xi == b):
            flag = 0
        
    # return the answer
    return x, y
    
    


### Perform the integration

In [ ]:
a = 0.0
b = 2.0 * np.pi

# initial conditions
y_0 = np.zeros(2)  # array of size 2
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

# perform the integration 
x,y = rk4_mv(dydx,a,b,y_0,tolerance)